# Creating a basic bot using PyTorch

# 1)- Importing key Modules

In [1]:
import numpy as np
import random
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from nltk_utils import bag_of_words, tokenize, stem
from model import NeuralNet

# 2-Load Data

In [2]:
with open('intents.json', 'r') as f:
    data = json.load(f)

In [3]:
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'Hey',
    'How are you',
    'Is anyone there?',
    'Hello',
    'Good day'],
   'responses': ['Hey :-)',
    'Hello, thanks for visiting',
    'Hi there, what can I do for you?',
    'Hi there, how can I help?']},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful", "Thank's a lot!"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'items',
   'patterns': ['Which items do you have?',
    'What kinds of items are there?',
    'What do you sell?'],
   'responses': ['We sell coffee and tea', 'We have coffee and tea']},
  {'tag': 'payments',
   'patterns': ['Do you take credit cards?',
    'Do you accept Mastercard?',
    'Can I pay with Paypal?',
    'Are you cash only?'],
   'responses': ['We 

In [4]:
data['intents']

[{'tag': 'greeting',
  'patterns': ['Hi',
   'Hey',
   'How are you',
   'Is anyone there?',
   'Hello',
   'Good day'],
  'responses': ['Hey :-)',
   'Hello, thanks for visiting',
   'Hi there, what can I do for you?',
   'Hi there, how can I help?']},
 {'tag': 'goodbye',
  'patterns': ['Bye', 'See you later', 'Goodbye'],
  'responses': ['See you later, thanks for visiting',
   'Have a nice day',
   'Bye! Come back again soon.']},
 {'tag': 'thanks',
  'patterns': ['Thanks', 'Thank you', "That's helpful", "Thank's a lot!"],
  'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
 {'tag': 'items',
  'patterns': ['Which items do you have?',
   'What kinds of items are there?',
   'What do you sell?'],
  'responses': ['We sell coffee and tea', 'We have coffee and tea']},
 {'tag': 'payments',
  'patterns': ['Do you take credit cards?',
   'Do you accept Mastercard?',
   'Can I pay with Paypal?',
   'Are you cash only?'],
  'responses': ['We accept VISA, Mastercard and Paypal',
   '

In [5]:
data['intents']

[{'tag': 'greeting',
  'patterns': ['Hi',
   'Hey',
   'How are you',
   'Is anyone there?',
   'Hello',
   'Good day'],
  'responses': ['Hey :-)',
   'Hello, thanks for visiting',
   'Hi there, what can I do for you?',
   'Hi there, how can I help?']},
 {'tag': 'goodbye',
  'patterns': ['Bye', 'See you later', 'Goodbye'],
  'responses': ['See you later, thanks for visiting',
   'Have a nice day',
   'Bye! Come back again soon.']},
 {'tag': 'thanks',
  'patterns': ['Thanks', 'Thank you', "That's helpful", "Thank's a lot!"],
  'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
 {'tag': 'items',
  'patterns': ['Which items do you have?',
   'What kinds of items are there?',
   'What do you sell?'],
  'responses': ['We sell coffee and tea', 'We have coffee and tea']},
 {'tag': 'payments',
  'patterns': ['Do you take credit cards?',
   'Do you accept Mastercard?',
   'Can I pay with Paypal?',
   'Are you cash only?'],
  'responses': ['We accept VISA, Mastercard and Paypal',
   '

In [7]:
all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in data['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w) # notice use of extend instead of append
        # add to xy pair
        xy.append((w, tag))

print(all_words)


['Hi', 'Hey', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hello', 'Good', 'day', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Thanks', 'Thank', 'you', 'That', "'s", 'helpful', 'Thank', "'s", 'a', 'lot', '!', 'Which', 'items', 'do', 'you', 'have', '?', 'What', 'kinds', 'of', 'items', 'are', 'there', '?', 'What', 'do', 'you', 'sell', '?', 'Do', 'you', 'take', 'credit', 'cards', '?', 'Do', 'you', 'accept', 'Mastercard', '?', 'Can', 'I', 'pay', 'with', 'Paypal', '?', 'Are', 'you', 'cash', 'only', '?', 'How', 'long', 'does', 'delivery', 'take', '?', 'How', 'long', 'does', 'shipping', 'take', '?', 'When', 'do', 'I', 'get', 'my', 'delivery', '?', 'Tell', 'me', 'a', 'joke', '!', 'Tell', 'me', 'something', 'funny', '!', 'Do', 'you', 'know', 'a', 'joke', '?']


This is like our vocab.

In [8]:
# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort. Set does trick of getting rid of all duplicated ones
all_words = sorted(set(all_words))
tags = sorted(set(tags))

In [9]:
print(len(xy), "patterns")

26 patterns


In [10]:
print(len(tags), "tags:", tags)

7 tags: ['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']


In [11]:
print(len(all_words), "unique stemmed words:", all_words)

54 unique stemmed words: ["'s", 'a', 'accept', 'anyon', 'are', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'deliveri', 'do', 'doe', 'funni', 'get', 'good', 'goodby', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'is', 'item', 'joke', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'my', 'of', 'onli', 'pay', 'paypal', 'see', 'sell', 'ship', 'someth', 'take', 'tell', 'thank', 'that', 'there', 'what', 'when', 'which', 'with', 'you']


In [12]:
# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [13]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 1.]], dtype=float32)

In [14]:
y_train

array([3, 3, 3, 3, 3, 3, 2, 2, 2, 6, 6, 6, 6, 4, 4, 4, 5, 5, 5, 5, 0, 0,
       0, 1, 1, 1])

In [15]:
print(X_train.shape)
print(y_train.shape)

(26, 54)
(26,)


In [16]:
# Hyper-parameters 
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size) 
print(output_size)

54
7


- Input beig X and its feature 

- y being number of classes we have

In [17]:
class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)


In [18]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [19]:
# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(device=device,dtype=torch.int64)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 1.6805
Epoch [200/1000], Loss: 0.6749
Epoch [300/1000], Loss: 0.2385
Epoch [400/1000], Loss: 0.2431
Epoch [500/1000], Loss: 0.0082
Epoch [600/1000], Loss: 0.0111
Epoch [700/1000], Loss: 0.0005
Epoch [800/1000], Loss: 0.0016
Epoch [900/1000], Loss: 0.0009
Epoch [1000/1000], Loss: 0.0009
final loss: 0.0009


In [20]:
data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

training complete. file saved to data.pth
